In [1]:
pip install -U liwc

In [26]:
#reinstalling b/c svm import issue
install scikit-learn

SyntaxError: invalid syntax (<ipython-input-26-7a727394c3fd>, line 2)

In [80]:
import pandas as pd
import numpy as np
import re
from collections import Counter
import collections
import liwc
import nltk
nltk.download('punkt')
nltk.download('wordnet')
from nltk.tokenize import word_tokenize
from nltk.tokenize import sent_tokenize
from nltk.stem.wordnet import WordNetLemmatizer

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [5]:
# load Reddit data
df_r = pd.read_csv('500_Reddit_users_posts_labels.csv')
# data already split into 3 columns
df_r.head()

,User,Post,Label
0,user-0,"['Its not a viable option, and youll be leavin...",Supportive
1,user-1,['It can be hard to appreciate the notion that...,Ideation
2,user-2,"['Hi, so last night i was sitting on the ledge...",Behavior
3,user-3,['I tried to kill my self once and failed badl...,Attempt
4,user-4,['Hi NEM3030. What sorts of things do you enjo...,Ideation


In [6]:
df_r.info()
# Columns with mixed types = object

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   User    500 non-null    object
 1   Post    500 non-null    object
 2   Label   500 non-null    object
dtypes: object(3)
memory usage: 11.8+ KB


In [7]:
df_r["Post"] = df_r["Post"].apply(lambda x: x.lstrip("['").rstrip("']"))
# original data read as list, need to remove [] and single quote
df_r['Post']

0      Its not a viable option, and youll be leaving ...
1      It can be hard to appreciate the notion that y...
2      Hi, so last night i was sitting on the ledge o...
3      I tried to kill my self once and failed badly ...
4      Hi NEM3030. What sorts of things do you enjoy ...
                             ...                        
495    Its not the end, it just feels that way. Or at...
496    It was a skype call, but she ended it and Vent...
497    That sounds really weird.Maybe you were Distra...
498    Dont know there as dumb as it sounds I feel Hy...
499    &gt;It gets better, trust me.Ive spent long en...
Name: Post, Length: 500, dtype: object

In [8]:
#sentence tokenize
#  df_r['tokenized_post'] = df_r['Post'].apply(lambda x: nltk.sent_tokenize)
df_r['sent_tokens'] = df_r['Post'].apply(sent_tokenize)
print(df_r['sent_tokens'])
#https://stackoverflow.com/questions/44173624/how-to-apply-nltk-word-tokenize-library-on-a-pandas-dataframe-for-twitter-data
# vs using punkt https://stackoverflow.com/questions/35275001/use-of-punktsentencetokenizer-in-nltk

0      [Its not a viable option, and youll be leaving...
1      [It can be hard to appreciate the notion that ...
2      [Hi, so last night i was sitting on the ledge ...
3      [I tried to kill my self once and failed badly...
4      [Hi NEM3030., What sorts of things do you enjo...
                             ...                        
495    [Its not the end, it just feels that way., Or ...
496    [It was a skype call, but she ended it and Ven...
497    [That sounds really weird.Maybe you were Distr...
498    [Dont know there as dumb as it sounds I feel H...
499    [&gt;It gets better, trust me.Ive spent long e...
Name: sent_tokens, Length: 500, dtype: object


In [9]:
#lowercase in numpy https://docs.scipy.org/doc/numpy/reference/generated/numpy.char.lower.html#numpy.char.lower
df_r['sent_tokens'] = [np.char.lower(x) for x in df_r['sent_tokens']]
print (df_r['sent_tokens'])

#remove list []
df_r['sent_tokens'] = [','.join(map(str, l)) for l in df_r['sent_tokens']]

0      [its not a viable option, and youll be leaving...
1      [it can be hard to appreciate the notion that ...
2      [hi, so last night i was sitting on the ledge ...
3      [i tried to kill my self once and failed badly...
4      [hi nem3030., what sorts of things do you enjo...
                             ...                        
495    [its not the end, it just feels that way., or ...
496    [it was a skype call, but she ended it and ven...
497    [that sounds really weird.maybe you were distr...
498    [dont know there as dumb as it sounds i feel h...
499    [&gt;it gets better, trust me.ive spent long e...
Name: sent_tokens, Length: 500, dtype: object


In [10]:
#word tokens
df_r['word_tokens'] = df_r['sent_tokens'].apply(word_tokenize)
print(df_r['word_tokens'])

#remove list []
df_r['word_tokens'] = [','.join(map(str, l)) for l in df_r['word_tokens']]

0      [its, not, a, viable, option, ,, and, youll, b...
1      [it, can, be, hard, to, appreciate, the, notio...
2      [hi, ,, so, last, night, i, was, sitting, on, ...
3      [i, tried, to, kill, my, self, once, and, fail...
4      [hi, nem3030., ,, what, sorts, of, things, do,...
                             ...                        
495    [its, not, the, end, ,, it, just, feels, that,...
496    [it, was, a, skype, call, ,, but, she, ended, ...
497    [that, sounds, really, weird.maybe, you, were,...
498    [dont, know, there, as, dumb, as, it, sounds, ...
499    [&, gt, ;, it, gets, better, ,, trust, me.ive,...
Name: word_tokens, Length: 500, dtype: object


In [11]:
wnl = WordNetLemmatizer()
df_r['lemma_post'] = df_r['sent_tokens'].apply(lambda lst:[wnl.lemmatize(word) for word in lst])
#https://stackoverflow.com/questions/61987040/how-to-lemmatise-a-dataframe-column-python

# df_r['lemma_post'] = df_r['word_tokens'].apply(lambda row: [wnl.lemmatize(i) for i in row])
#https://stackoverflow.com/questions/74853779/lemmatize-a-list-of-tokens-in-dataframe

print(df_r['lemma_post'])

# https://stackoverflow.com/questions/45306988/column-of-lists-convert-list-to-string-as-a-new-column
# df_r['lemma_post'] = [','.join(map(str, l)) for l in df_r['lemma_post']]

0      [i, t, s,  , n, o, t,  , a,  , v, i, a, b, l, ...
1      [i, t,  , c, a, n,  , b, e,  , h, a, r, d,  , ...
2      [h, i, ,,  , s, o,  , l, a, s, t,  , n, i, g, ...
3      [i,  , t, r, i, e, d,  , t, o,  , k, i, l, l, ...
4      [h, i,  , n, e, m, 3, 0, 3, 0, ., ,, w, h, a, ...
                             ...                        
495    [i, t, s,  , n, o, t,  , t, h, e,  , e, n, d, ...
496    [i, t,  , w, a, s,  , a,  , s, k, y, p, e,  , ...
497    [t, h, a, t,  , s, o, u, n, d, s,  , r, e, a, ...
498    [d, o, n, t,  , k, n, o, w,  , t, h, e, r, e, ...
499    [&, g, t, ;, i, t,  , g, e, t, s,  , b, e, t, ...
Name: lemma_post, Length: 500, dtype: object


In [12]:
from sklearn import feature_extraction


IRRELEVANT_CATEGORIES = {"funct", "we", "you", "shehe", "article", "verb",
                         "auxverb", "past", "present", "future", "adverb", "preps", "conj", "quant",
                         "number", "swear", "social", "family", "friend", "humans", "affect",
                         "posemo", "anx", "cogmech", "insight", "cause", "discrep", "tentat",
                         "certain", "inhib", "incl", "excl", "percept", "see", "hear", "bio",
                        "health", "sexual", "ingest", "relativ", "motion", "space", "time",
                         "work", "achieve", "leisure", "home", "money", "relig", "assent",
                         "filler", "ipron"} #impersonal pronoun

# sentence and word tokenize at same time instead of performing separately
#focuses data/matrix to features (not Y axis info/label)
def liwc_features(post, lemmatizer, parse):
    liwc_counts = collections.Counter()
    for sentence in nltk.sent_tokenize(post):
        for word in nltk.word_tokenize(sentence):
            word = word.casefold()
            lemma = lemmatizer.lemmatize(word)
            for category in parse(lemma):
                # ignore particular LIWC features
                if category in IRRELEVANT_CATEGORIES:
                    continue
                liwc_counts[category] += 1
    return liwc_counts

lemmatizer = nltk.WordNetLemmatizer()

parse, category_names = liwc.load_token_parser("liwc.dic")

In [ ]:
for user, post in zip(df_r['User'], df_r['Post']):
  print(user, liwc_features(post, lemmatizer, parse))

user-0 Counter({'pronoun': 24, 'ppron': 14, 'negemo': 6, 'negate': 3, 'anger': 2, 'i': 2, 'body': 2, 'feel': 2, 'sad': 1, 'they': 1})
user-1 Counter({'pronoun': 398, 'ppron': 242, 'negemo': 58, 'negate': 54, 'they': 43, 'i': 43, 'feel': 23, 'sad': 17, 'anger': 14, 'death': 8, 'body': 7, 'nonfl': 4})
user-2 Counter({'pronoun': 98, 'ppron': 52, 'i': 21, 'negate': 15, 'negemo': 6, 'death': 3, 'feel': 2, 'sad': 2, 'body': 1, 'they': 1})
user-3 Counter({'pronoun': 182, 'ppron': 122, 'i': 85, 'negemo': 26, 'feel': 14, 'negate': 13, 'sad': 8, 'anger': 7, 'death': 6, 'body': 5, 'nonfl': 2, 'they': 1})
user-4 Counter({'pronoun': 37, 'ppron': 29, 'i': 15, 'negate': 7, 'negemo': 5, 'sad': 4, 'death': 3, 'body': 2, 'feel': 2, 'anger': 1, 'nonfl': 1})
user-5 Counter({'pronoun': 540, 'ppron': 349, 'i': 127, 'negemo': 69, 'negate': 38, 'feel': 30, 'they': 15, 'anger': 15, 'sad': 10, 'death': 9, 'body': 5, 'nonfl': 1})
user-6 Counter({'pronoun': 238, 'ppron': 158, 'i': 43, 'negate': 41, 'negemo': 21, 

In [13]:
# extract patterns from .txt files
diagnose_lang = "diagpatterns_positive.txt"
mh_patterns = "mh_patterns.txt"
dl_list = []
mh_list = []

#remove new line (rstrip), append to list-- use list instead of opening file each time
#2 lists-- strip new line
with open(diagnose_lang, 'r') as d, open(mh_patterns, 'r') as mh:
  for line_d in d:
    clean_dl = line_d.rstrip()
    dl_list.append(clean_dl)

  for line_mh in mh:
    clean_mh = line_mh.rstrip()
    mh_list.append(clean_mh)

print(dl_list)
print(mh_list)
#zip helpful for equal lengths

['i am diagnosed with', 'i got diagnos', 'i just got diagnos', 'i have been diagnos', 'i was diagnos', 'i have a diagnos', 'i have diagnos', "i've been diagnos", 'i suffer from clinically diagnos', 'i suffer from diagnos', "i've been diagnosed with", 'my doctor diagnosed me', 'she diagnosed me', 'he diagnosed me', 'they diagnosed me', 'recently diagnosed me as', 'i was recently diagnos', 'i have recently been diagnos', 'i have been recently diagnos', 'until i was diagnosed this year', 'last year i was diagnos', 'i have honestly diagnosed', 'i have been recently diagnos', 'i have recently been diagnos', 'i was recently diagnos', 'they diagnosed me with', 'as someone with a diagnos', 'im diagnos', "i'm diagnos", 'i was _doctor diagnos', "i've been _doctor diagnos", 'i have been _doctor diagnos', 'i was just diagnos', "i've been recently diagnos", "i've recently been diagnos", "i've just been diagnos", "i've been diagnos", "i've only recently been diagnos", "i've only been diagnos", '_doc

In [ ]:
df_r.head()

,User,Post,Label,tokenized_post,word_tokens,lemma_post,Comorbidity
0,user-0,"Its not a viable option, and youll be leaving ...",Supportive,"[Its not a viable option, and youll be leaving...","[its, not, a, viable, option, ,, and, youll, b...","[it, not, a, viable, option, ,, and, youll, be...",False
1,user-1,It can be hard to appreciate the notion that y...,Ideation,[It can be hard to appreciate the notion that ...,"[it, can, be, hard, to, appreciate, the, notio...","[it, can, be, hard, to, appreciate, the, notio...",True
2,user-2,"Hi, so last night i was sitting on the ledge o...",Behavior,"[Hi, so last night i was sitting on the ledge ...","[hi, ,, so, last, night, i, was, sitting, on, ...","[hi, ,, so, last, night, i, wa, sitting, on, t...",False
3,user-3,I tried to kill my self once and failed badly ...,Attempt,[I tried to kill my self once and failed badly...,"[i, tried, to, kill, my, self, once, and, fail...","[i, tried, to, kill, my, self, once, and, fail...",True
4,user-4,Hi NEM3030. What sorts of things do you enjoy ...,Ideation,"[Hi NEM3030., What sorts of things do you enjo...","[hi, nem3030., ,, what, sorts, of, things, do,...","[hi, nem3030., ,, what, sort, of, thing, do, y...",True


In [ ]:
df_r.Label.value_counts(normalize=True)
# breakdown of labels

Label
Ideation      0.342
Supportive    0.216
Indicator     0.198
Behavior      0.154
Attempt       0.090
Name: proportion, dtype: float64

In [14]:
#gives yes/no comorbidity present
comorbidity = []
for user, post in zip(df_r['User'], df_r['Post']):
  # print(user, post.partition("diagnosed")[2])
  value = post.partition("diagnosed")[2]
  # print(value)
  found = False
  ##depres* for regex (better groups) instead of mh_list w/ multiple spellings
  for mhline in mh_list: #not matching num in list
    match = re.search(re.escape(mhline), post, re.IGNORECASE) #search finds first match
    if match:
      found = True
      break
  comorbidity.append(found)

#if any matches, quit loop
print(comorbidity)
len(comorbidity)

[False, True, False, True, True, False, True, True, False, False, False, True, False, True, False, True, True, False, True, True, False, True, True, False, True, True, True, False, True, False, True, False, True, True, True, True, True, True, False, True, False, False, False, True, True, True, True, False, True, True, False, True, False, True, True, False, False, True, False, True, True, False, True, True, True, True, False, False, False, False, True, True, True, True, False, True, False, True, False, True, False, False, False, False, True, True, True, True, False, True, False, True, False, False, True, True, False, True, True, True, True, True, True, False, False, True, False, True, True, False, True, False, False, True, True, False, True, True, False, False, True, False, True, True, False, True, True, True, True, False, False, True, False, True, True, False, True, True, True, False, True, True, True, True, True, False, True, True, True, False, True, True, False, True, False, True, Fa

500

In [15]:
#adds true/false labels to dataframe = presence of comorbidity
df_r['Comorbidity'] = comorbidity

In [ ]:
# mh patterns, multiple matches within 1 user
for user, post in zip(df_r["User"], df_r["Post"]):
  for mhline in mh_list:
    match = re.search(re.escape(mhline), post, re.IGNORECASE)
    if match:
      print(user, match) #index for end of string given
      #extract after span (look at right of string)

user-1 <re.Match object; span=(2432, 2438), match='bipola'>
user-1 <re.Match object; span=(2432, 2439), match='bipolar'>
user-1 <re.Match object; span=(2432, 2439), match='bipolar'>
user-1 <re.Match object; span=(2432, 2439), match='bipolar'>
user-1 <re.Match object; span=(2874, 2882), match='bipolari'>
user-3 <re.Match object; span=(3144, 3151), match='Anxiety'>
user-3 <re.Match object; span=(2278, 2284), match='depres'>
user-3 <re.Match object; span=(2278, 2287), match='depressio'>
user-3 <re.Match object; span=(2278, 2288), match='depression'>
user-3 <re.Match object; span=(2278, 2288), match='depression'>
user-3 <re.Match object; span=(2278, 2289), match='depressions'>
user-3 <re.Match object; span=(2337, 2347), match='depressive'>
user-3 <re.Match object; span=(2337, 2347), match='depressive'>
user-4 <re.Match object; span=(330, 341), match='suffer from'>
user-6 <re.Match object; span=(1766, 1772), match='depres'>
user-6 <re.Match object; span=(1766, 1775), match='depressed'>
user

KeyboardInterrupt: 

experiment 1, predict comorbidity presence T/F (binary) from selected liwc features

In [16]:
# experiment 1, define data X Y
#bigger/X has more structure 2D (matrix, list of lists); y is usully single vector 1D (label)
vectorizer = feature_extraction.DictVectorizer(sparse=False)
X_liwc = vectorizer.fit_transform(liwc_features(post, lemmatizer, parse) for post in df_r['Post'])
#data/matrix

Y = df_r['Comorbidity'] #labels on data/column vector

In [20]:
# search feature normalization sklearn (zscale normalizing) https://scikit-learn.org/stable/auto_examples/preprocessing/plot_scaling_importance.html
from sklearn.preprocessing import StandardScaler
# Split data into testing and training
from sklearn.model_selection import train_test_split
# https://stackoverflow.com/questions/34842405/parameter-stratify-from-method-train-test-split-scikit-learn
#stratify makes a split so that the proportion of values in the sample produced will be the same as the proportion of values provided by parameter stratify
#... random_state=42, stratify=y)
#a fixed value will guarantee that same sequence of random numbers are generated each time you run the code
#https://stackoverflow.com/questions/42191717/scikit-learn-random-state-in-splitting-dataset

# scaler = StandardScaler().set_output(transform="pandas")
scaler = StandardScaler()

Xtrain, Xtest, Ytrain, Ytest = train_test_split(
    X_liwc, Y, test_size=0.30, random_state=42)

scaled_Xtrain = scaler.fit_transform(Xtrain)
# scaled_Ytrain = scaler.fit_transform(Ytrain)

In [101]:
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, f1_score
from sklearn.metrics import classification_report
import sklearn.metrics as metrics
# from sklearn import metrics
from sklearn.model_selection import KFold, cross_val_score

#pipeline to scale data & initialize model
LR = LogisticRegression()
LRpipe = make_pipeline(StandardScaler(), LogisticRegression())
#fitting to model
LRpipe.fit(Xtrain, Ytrain)  # apply scaling on training data

# predicting values
LRY_pred = LRpipe.predict(Xtest)

# calculating accuracy score
LRaccuracy_score = accuracy_score(LRY_pred,Ytest)
print("LR Model Accuracy:",LRaccuracy_score)

LRpipe.score(Xtest, Ytest)

#f1 score
LRf1_score = f1_score(Ytest, LRY_pred, average='macro')
print("LR macro F1 Score:",LRf1_score)
# print('macro F-1 score : {}'.format(np.round(LRf1_score,5)))
#https://www.geeksforgeeks.org/make_pipeline-function-in-sklearn/

#cross validation score
k_folds = KFold(n_splits = 5)
LRCV_scores = cross_val_score(LR, X_liwc, Y, cv = k_folds)
print("cross validation score:",LRCV_scores)
# https://stackoverflow.com/questions/62658215/convergencewarning-lbfgs-failed-to-converge-status-1-stop-total-no-of-iter

# classification report
print(metrics.classification_report(Ytest, LRY_pred))

LR Model Accuracy: 0.7933333333333333
LR macro F1 Score: 0.7799441578723203
cross validation score: [0.79 0.8  0.8  0.83 0.81]
              precision    recall  f1-score   support

       False       0.65      0.82      0.73        50
        True       0.90      0.78      0.83       100

    accuracy                           0.79       150
   macro avg       0.77      0.80      0.78       150
weighted avg       0.81      0.79      0.80       150



/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _c

In [44]:
from sklearn import svm

svmpipe = make_pipeline(StandardScaler(), svm.LinearSVC())
#fitting to model
svmpipe.fit(Xtrain, Ytrain)  # apply scaling on training data

# predicting values
svmY_pred = svmpipe.predict(Xtest)

# calculating accuracy score
svmaccuracy_score = accuracy_score(svmY_pred,Ytest)
# svmpipe.score(Xtest, Ytest)
print("Linear SVM Model Accuracy:",svmaccuracy_score)

#f1 score
svmf1_score = f1_score(Ytest, svmY_pred, average='macro')
print("SVM macro F1 Score:",svmf1_score)
# print('macro F-1 score : {}'.format(np.round(svmf1_score,5)))

# classification report
print(metrics.classification_report(Ytest, svmY_pred))

Linear SVM Model Accuracy: 0.78
SVM F1 Score: 0.7688104245481295
macro F-1 score : 0.76881
              precision    recall  f1-score   support

       False       0.63      0.84      0.72        50
        True       0.90      0.75      0.82       100

    accuracy                           0.78       150
   macro avg       0.77      0.79      0.77       150
weighted avg       0.81      0.78      0.79       150



In [45]:
import xgboost as xgb
# https://www.kaggle.com/code/stuarthallows/using-xgboost-with-scikit-learn

#binary task
xgbpipe = make_pipeline(StandardScaler(), xgb.XGBClassifier(objective="binary:logistic", random_state=42))
#fitting to model
xgbpipe.fit(Xtrain, Ytrain)  # apply scaling on training data

# predicting values
xgbY_pred = xgbpipe.predict(Xtest)

# calculating accuracy score
xgbaccuracy_score = accuracy_score(xgbY_pred,Ytest) #xgbpipe.score(Xtest, Ytest)
print("XGB Model Accuracy:",xgbaccuracy_score)

#f1 score
xgbf1_score = f1_score(Ytest, xgbY_pred, average='macro')
print("XGB macro F1 Score:",xgbf1_score)
# print('macro F-1 score : {}'.format(np.round(xgbf1_score,5)))

# classification report
print(metrics.classification_report(Ytest, xgbY_pred))

XGB Model Accuracy: 0.7666666666666667
XGB F1 Score: 0.7435897435897436
macro F-1 score : 0.74359
              precision    recall  f1-score   support

       False       0.64      0.70      0.67        50
        True       0.84      0.80      0.82       100

    accuracy                           0.77       150
   macro avg       0.74      0.75      0.74       150
weighted avg       0.77      0.77      0.77       150



In [65]:
# NOT USED
from sklearn.naive_bayes import MultinomialNB
from sklearn.preprocessing import MinMaxScaler
# https://stackoverflow.com/questions/67224279/how-can-i-resolve-this-error-valueerror-negative-values-in-data-passed-to-m
minmax_scaler = MinMaxScaler() #normalizes to the range 0 to 1 thus removing negative numbers
Xtrain_minmax = minmax_scaler.fit_transform(Xtrain)
Xtest_minmax = minmax_scaler.transform(Xtest)

# https://stackoverflow.com/questions/24169238/dealing-with-negative-values-in-sklearn-multinomialnb
# matrix factorization doesn't work for data b/c neg numbers
mnb_pipe = make_pipeline(MinMaxScaler(), MultinomialNB(alpha=.05))

mnbpipe = make_pipeline(StandardScaler(), MultinomialNB(alpha=.05))

#fitting to model
mnbpipe.fit(Xtrain_minmax, Ytrain)  # apply scaling on training data

# predicting values on test data
mnbY_pred = mnbpipe.predict(Xtest_minmax)

# calculating accuracy score
mnbaccuracy_score = accuracy_score(mnbY_pred,Ytest) #
print('MNB accuracy score : ',mnbaccuracy_score)


#https://stackoverflow.com/questions/24169238/dealing-with-negative-values-in-sklearn-multinomialnb

ValueError: Negative values in data passed to MultinomialNB (input X)

experiment 2, predict suicidality (multiclass) from selected liwc features

In [35]:
# experiment 2, define X Y (multiclass classification, return 1 predicted)
X_liwc = vectorizer.fit_transform(liwc_features(post, lemmatizer, parse) for post in df_r['Post'])
Y_sui = df_r['Label'] #suicidality label

#split data
X_train, X_test, Y_train, Y_test = train_test_split(
    X_liwc, Y_sui, test_size=0.30, random_state=42)

In [46]:
#pipeline to scale data & initialize model
#fitting to model
LRpipe.fit(X_train, Y_train)  # apply scaling on training data

# predicting values
LR_Y_pred = LRpipe.predict(X_test)

# calculating accuracy score
LR_accuracy_score = accuracy_score(LR_Y_pred,Y_test) # LR_pipe.score(X_test, Y_test)
print("LR Model Accuracy:",LR_accuracy_score)

#samples f1 score
LR_f1_score = f1_score(Y_test, LR_Y_pred, average='micro')
print("LR micro F1 Score:",LR_f1_score)
#https://datascience.stackexchange.com/questions/15989/micro-average-vs-macro-average-performance-in-a-multiclass-classification-settin/24051#24051
#multi-class classification setup, micro-average is preferable if you suspect there might be class imbalance
 #(i.e you may have many more examples of one class than of other classes).

# print('macro F-1 score : {}'.format(np.round(LR_f1_score,5)))
#https://stackoverflow.com/questions/55740220/macro-vs-micro-vs-weighted-vs-samples-f1-score
# weighted f1- compute f1 for each label, and
#returns the average considering the proportion for each label in the dataset.

# classification report
print(metrics.classification_report(Y_test, LR_Y_pred))

LR Model Accuracy: 0.24666666666666667
LR macro F1 Score: 0.13273267015763512
              precision    recall  f1-score   support

     Attempt       0.00      0.00      0.00        19
    Behavior       0.12      0.04      0.06        23
    Ideation       0.26      0.72      0.39        43
   Indicator       0.11      0.04      0.05        28
  Supportive       0.31      0.11      0.16        37

    accuracy                           0.25       150
   macro avg       0.16      0.18      0.13       150
weighted avg       0.19      0.25      0.17       150



In [38]:
#fitting to model
svmpipe.fit(X_train, Y_train)  # apply scaling on training data

# predicting values
svm_Y_pred = svmpipe.predict(X_test)

# calculating accuracy score
svm_accuracy_score = accuracy_score(svm_Y_pred,Y_test)
print("Linear SVM Model Accuracy:",svmaccuracy_score)

#f1 score
svm_f1_score = f1_score(Y_test, svm_Y_pred, average='micro')
print("SVM micro F1 Score:",svm_f1_score)

# classification report
print(metrics.classification_report(Y_test, svm_Y_pred))

Linear SVM Model Accuracy: 0.78
SVM F1 Score: 0.13055936377281843
macro F-1 score : 0.13056


/usr/local/lib/python3.10/dist-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


In [55]:
# NOT USED
from sklearn.naive_bayes import MultinomialNB
from sklearn.preprocessing import MinMaxScaler
# https://stackoverflow.com/questions/67224279/how-can-i-resolve-this-error-valueerror-negative-values-in-data-passed-to-m
min_scaler = MinMaxScaler() #normalizes to the range 0 to 1 thus removing negative numbers
min_X_train = min_scaler.fit_transform(X_train)
min_X_test = min_scaler.transform(X_test)
 X_train_minmax = min_max_scaler.fit_transform(X_train)
# https://stackoverflow.com/questions/24169238/dealing-with-negative-values-in-sklearn-multinomialnb
# matrix factorization doesn't work for data b/c neg numbers
mnb_pipe = make_pipeline(MinMaxScaler(), MultinomialNB(alpha=.05))

#fitting to model
mnb_pipe.fit(X_train, X_test)  # apply scaling on training data

# predicting values on test data
mnb_Y_pred = mnb_pipe.predict(min_X_test)

# calculating accuracy score
mnb_accuracy_score = accuracy_score(mnb_Y_pred,Y_test)
print('MNB accuracy score : ',mnb_accuracy_score)

#f1 score
mnb_f1_score = f1_score(Y_test, mnb_Y_pred, average='micro')
print("MNB micro F1 Score:",mnb_f1_score)

# classification report
print(metrics.classification_report(Y_test, mnb_Y_pred))

ValueError: y should be a 1d array, got an array of shape (150, 12) instead.

In [50]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
# https://stackoverflow.com/questions/71996617/invalid-classes-inferred-from-unique-values-of-y-expected-0-1-2-3-4-5-got
xgb_le_Y_train = le.fit_transform(Y_train)
xgb_le_Y_test = le.fit_transform(Y_test)
# labels should be numeric

#multiclass task
xgb_pipe = make_pipeline(StandardScaler(), xgb.XGBClassifier(objective="multi:softprob", random_state=42))
#fitting to model
xgb_pipe.fit(X_train, xgb_le_Y_train)  # apply scaling on training data

# predicting values
xgb_Y_pred = xgb_pipe.predict(X_test)

# calculating accuracy score
xgb_accuracy_score = accuracy_score(xgb_Y_pred,xgb_le_Y_test)
print("XGB Model Accuracy:",xgb_accuracy_score)

xgb_pipe.score(X_test, xgb_le_Y_test)

#f1 score
xgb_f1_score = f1_score(xgb_le_Y_test, xgb_Y_pred, average='micro')
print("XGB micro F1 Score:",xgb_f1_score)

# classification report
print(metrics.classification_report(xgb_le_Y_test, xgb_Y_pred))

XGB Model Accuracy: 0.28
XGB macro F1 Score: 0.2175001423636298
              precision    recall  f1-score   support

           0       0.00      0.00      0.00        19
           1       0.11      0.09      0.10        23
           2       0.29      0.47      0.36        43
           3       0.19      0.18      0.18        28
           4       0.50      0.41      0.45        37

    accuracy                           0.28       150
   macro avg       0.22      0.23      0.22       150
weighted avg       0.26      0.28      0.26       150



In [83]:
# statistical analysis of features w/ suicidality & cormorbidity as independent features
#how many w/ label has diagnosed mh
#associated w/ mental health pattern? simple LR, statistical results, get p value -- work different from prediction

#statistically correlated? reason #may help choose features, separate work from classifier

comor_le = le.fit_transform(df_r['Comorbidity'])

experiment 3, predict suicidality (multiclass) from selected liwc features & comor T/F labels

In [92]:
# experiment 3
from sklearn.pipeline import Pipeline, FeatureUnion
#https://stackoverflow.com/questions/56774862/how-to-add-a-feature-using-a-pipeline-and-featureunion

# X_combine = FeatureUnion([
#       # added feature
#       (df_r['Comorbidity']),
#       # previous features
#       (X_liwc)])

X_liwc_comor = vectorizer.fit_transform(liwc_features(post, lemmatizer, parse) for post in df_r['Post'])
#data/matrix

# Xcombine = X_liwc_comor.numpy.append(df_r['Comorbidity']) #include as feature (append) LIWC&comord

Xcombine = np.concatenate((X_liwc_comor, comor_le[:, None]), axis=1)
#concat along axis 1

Y_sui = df_r['Label'] #suicidality label

In [94]:
X_combinetrain, X_combinetest, Y_train, Y_test = train_test_split(
    Xcombine, Y_sui, test_size=0.30, random_state=42)

In [1]:
#fitting to model
LRpipe.fit(X_combinetrain, Y_train)  # apply scaling on training data

# predicting values
combine_LR_Y_pred = LRpipe.predict(X_combinetest)

# calculating accuracy score
combine_LR_accuracy_score = accuracy_score(combine_LR_Y_pred,Y_test)
print("LR Model Accuracy:",combine_LR_accuracy_score)

#f1 score
combine_LR_f1_score = f1_score(Y_test, combine_LR_Y_pred, average='micro')
print("LR micro F1 Score:",combine_LR_f1_score)

# classification report
print(metrics.classification_report(Y_test, combine_LR_Y_pred))

NameError: name 'make_pipeline' is not defined

In [ ]:
#fitting to model
svmpipe.fit(X_combinetrain, Y_train)  # apply scaling on training data

# predicting values
combine_svm_Y_pred = svmpipe.predict(X_combinetest)

# calculating accuracy score
combine_svm_accuracy_score = accuracy_score(combine_svm_Y_pred,Y_test)
print("Linear SVM Model Accuracy:",combine_svm_accuracy_score)

#f1 score
combine_svm_f1_score = f1_score(Y_test, combine_svm_Y_pred, average='micro')
print("SVM micro F1 Score:",combine_svm_f1_score)

# classification report
print(metrics.classification_report(Y_test, combine_svm_Y_pred))

In [ ]:
xgb_le_Y_train = le.fit_transform(Y_train)
xgb_le_Y_test = le.fit_transform(Y_test)
# labels should be numeric

#multiclass task
xgb_pipe = make_pipeline(StandardScaler(), xgb.XGBClassifier(objective="multi:softprob", random_state=42))
#fitting to model
xgb_pipe.fit(X_combinetrain, xgb_le_Y_train)  # apply scaling on training data

# predicting values
combine_xgb_Y_pred = xgb_pipe.predict(X_combinetest)

# calculating accuracy score
combine_xgb_accuracy_score = accuracy_score(combine_xgb_Y_pred,xgb_le_Y_test)
print("XGB Model Accuracy:",combine_xgb_accuracy_score)

xgb_pipe.score(X_combinetest, xgb_le_Y_test)

#f1 score
combine_xgb_f1_score = f1_score(xgb_le_Y_test, xgb_Y_pred, average='micro')
print("XGB micro F1 Score:",combine_xgb_f1_score)

# classification report
print(metrics.classification_report(xgb_le_Y_test, combine_xgb_Y_pred))

In [ ]:
# multitask classifier: performs better when predicting cormorbidity + suicidality vs predicting suicidality alone?
#same data, train system for multiple tasks- diff domains --> subfield of deep learning
# want output vector size of 2 (layers)

#https://scikit-learn.org/stable/modules/multiclass.html#multiclass-multioutput-classification
from sklearn.datasets import make_classification
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.utils import shuffle

x = vectorizer.fit_transform(liwc_features(post, lemmatizer, parse) for post in df_r['Post'])
y = df_r[['Label', 'Comorbidity']].values
x, y = make_classification(n_samples=10, n_features=100,
                            n_informative=30, n_classes=3,
                            random_state=1)
# Split the data into training and testing sets
xtrain, xtest, ytrain, ytest = train_test_split(
    x, y, test_size=0.2, random_state=42)

mo_clf = MultiOutputClassifier()

# Train the model
mo_clf.fit(xtrain, ytrain)

# Make predictions on the test set
predictions = mo_clf.predict(xtest)

# Evaluate the performance
mse = mean_squared_error(ytest, predictions)
print(f'Mean Squared Error: {mse}')


y2 = shuffle(y1, random_state=1)
y3 = shuffle(y1, random_state=2)
Y = np.vstack((y1, y2, y3)).T
n_samples, n_features = X.shape
n_outputs = Y.shape[1] # 3
n_classes = 3
forest = RandomForestClassifier(random_state=1)
multi_target_forest = MultiOutputClassifier(forest, n_jobs=2)
multi_target_forest.fit(X, Y).predict(X)



In [ ]:
# multioutput classifier to use same X for predicting different Ys
#https://calmcode.io/course/scikit-meta/multi-output

Y_labels = df_r[['Label', 'Comorbidity']].values
X_out = vectorizer.fit_transform(liwc_features(post, lemmatizer, parse) for post in df_r['Post'])

clf = MultiOutputClassifier(LogisticRegression()).fit(X_out, Y_labels)
clf.predict(X_out)

In [ ]:
clf.predict_proba(X_out)
#first array predicting label; second predicting comor

improve baseline for experiment 2 & 3

In [ ]:
# higher suicidality = more use of I, death, personal pronoun, they, negate
# cormorbidity = more categories: body, sad, anger, negemo, nonfl, feel

#In this case, the features will be the categories you listed.
#The value of each feature will be the count of words in that category (divided by the total word count). You can experiment with the full feature set (all category counts) versus the handcrafted set you enumerated.
#Those would be two different experimental conditions.

# classifier: performs better at predicting suicidality when cormorbidity included as

In [ ]:
# try experiment 2 & 3 w/ all liwc categories
IRRELEVANT_CATEGORIES = {}

# sentence and word tokenize at same time instead of performing separately
#focuses data/matrix to features (not Y axis info/label)
def all_liwc_features(post, lemmatizer, parse):
    liwc_counts = collections.Counter()
    for sentence in nltk.sent_tokenize(post):
        for word in nltk.word_tokenize(sentence):
            word = word.casefold()
            lemma = lemmatizer.lemmatize(word)
            for category in parse(lemma):
                # ignore particular LIWC features
                if category in IRRELEVANT_CATEGORIES:
                    continue
                liwc_counts[category] += 1
    return liwc_counts

lemmatizer = nltk.WordNetLemmatizer()

parse, category_names = liwc.load_token_parser("liwc.dic")

In [ ]:
#experiment 2, define X Y (multiclass classification, return 1 predicted)
X_all_liwc = vectorizer.fit_transform(all_liwc_features(post, lemmatizer, parse) for post in df_r['Post'])

Y_sui = df_r['Label'] #suicidality label

#split data
X_all_train, X_all_test, Y_train, Y_test = train_test_split(
    X_all_liwc, Y_sui, test_size=0.30, random_state=42)

In [ ]:
#fitting to model
LR_pipe.fit(X_all_train, Y_train)  # apply scaling on training data

# predicting values
LRall_Y_pred = LR_pipe.predict(X_all_test)

# calculating accuracy score
LRall_accuracy_score = accuracy_score(LRall_Y_pred,Y_test) # LR_pipe.score(X_test, Y_test)
print("LR Model Accuracy:",LR_accuracy_score)

#samples f1 score
LRall_f1_score = f1_score(Y_test, LRall_Y_pred, average='micro')
print("LR micro F1 Score:",LRall_f1_score)

# classification report
print(metrics.classification_report(Y_test, LRall_Y_pred))